In [47]:
import torch
import torch.nn as nn

import nltk
import numpy as np
import pandas as pd
nltk.download('punkt_tab')

from nltk.stem.porter import PorterStemmer

import json,urllib
from torch.utils.data import Dataset, DataLoader

import random

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [48]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out

In [49]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out

In [50]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):

    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype = np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

In [51]:
url = '/content/intents.json'
with open(url, 'r') as f:
    intents = json.load(f)


In [52]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?','!','.',',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags  = sorted(set(tags))

X_train  = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # CrossEntropyLoss

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # dataset[idx]
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyper paramters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000


dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size,
                        shuffle =True, num_workers=2)

device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        # f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}'
        print("epoch {}/{}, loss={:.4f}.".format(epoch+1,num_epochs,loss.item()))

# print(f'Final loss, loss={loss.item():.4f}')
print("Final Loss, loss{:.4f}".format(loss.item()))

epoch 100/1000, loss=0.8387.
epoch 200/1000, loss=0.0983.
epoch 300/1000, loss=0.0508.
epoch 400/1000, loss=0.0111.
epoch 500/1000, loss=0.0041.
epoch 600/1000, loss=0.0014.
epoch 700/1000, loss=0.0016.
epoch 800/1000, loss=0.0005.
epoch 900/1000, loss=0.0013.
epoch 1000/1000, loss=0.0002.
Final Loss, loss0.0002


In [53]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hiddent_size": hidden_size,
    "all_words":all_words,
    "tags": tags
}
FILE = "data.pth"
torch.save(data, FILE)

print("Traning complete. file saved to",FILE)

Traning complete. file saved to data.pth


In [54]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

with open(url,'r') as f:
  intents = json.load(f)


FILE = "data.pth"
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hiddent_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

cpu


NeuralNet(
  (l1): Linear(in_features=80, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=12, bias=True)
  (relu): ReLU()
)

In [61]:
df = pd.read_csv("/content/all_india_PO_list_without_APS_offices_ver2_lat_long.csv")

In [62]:
df.columns


Index(['officename', 'pincode', 'officeType', 'Deliverystatus', 'divisionname',
       'regionname', 'circlename', 'Taluk', 'Districtname', 'statename',
       'Telephone', 'Related Suboffice', 'Related Headoffice', 'longitude',
       'latitude'],
      dtype='object')

In [63]:
df = df[['officename','pincode']]
df.head()

,officename,pincode
0,Achalapur B.O,504273
1,Ada B.O,504293
2,Adegaon B.O,504307
3,Adilabad Collectorate S.O,504001
4,Adilabad H.O,504001


In [58]:
def callme():
  po = {v: k for v, k in enumerate(pf)}
  print("AJ: Select your near Post Office \n ")

  for i,j in po.items():
    print(i,j)

  sel = int(input("\n Enter Number"))
  print("\n Taru: You selected {} Post office. \n".format(po[sel]))

  sen = input("AJ: Enter another query \n You: ")
  return sen

In [60]:
bot_name = "AJ"

print("Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 1234567890.")
while True:
  sentence = input('You: ')

  for word in sentence.split():
    if word.isdigit():
      word = int(word)
      pf = list(df['Office Name'][df['Pincode'] == word ])
      if len(pf) == 0:
        print("Sorry to inform you, We don't deliver here :(")
        break
      sentence = callme()
      # sentence = input('You: ')

    # else:
      # print("Please Enter Pincode !!!")

  if sentence =='quit':
    break

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]


  if prob.item() >0.75:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(bot_name, ":",random.choice(intent['responses']))
        # print("{}:{}".format(bot_name,random.choice(intent['responses'])))

  else:
    print("{} I do not understand...".format(bot_name))

Namste !!! We are working everywhere in Pan-india. Just drop you delivery location pincode/postal code to check delivery availability or ask us at Whatsapp Helpline 1234567890.
You: hi
AJ : Hi there, what can I do for you?
You: who are you
AJ I do not understand...
You: Tell me joke
AJ : I can’t believe I got fired from the calendar factory. All I did was take a day off!
You: What's item do you have, What on the menu
AJ : We have Cakes, Flowers and Gifts are on the menu!
You: Do you take credit card, do you accept cash
AJ : We accept most major Credit cards, Master Card, Paypal, American Express and Discover.
You: thanks
AJ : My Pleasure
You: quit
